In [1]:
%matplotlib inline

# Dynamic SAX tutorial
We assume there that you already know what is a SAX transformation. If you don't, please refer to our [section about it](https://timeseriesanalysis.github.io/static_sax.html) or on our [tutorial](https://timeseriesanalysis.github.io/static_sax_tutorial.html).
The aim of this tutorial is to show how works our algorithm on simple dynamic data first and then on dynamic EEG data.
The following examples are leaning on __incremental_SAX__ file in __script__ folder of our repository.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from scipy.stats import norm
sys.path.append("..")
from script.incremental_SAX import Incremental_SAX